In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110256 

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,167608.4871
2018-02-28,119755.7206
2018-03-31,161688.8548
2018-04-30,164314.2814
2018-05-31,143276.7166
2018-06-30,168584.7850
2018-07-31,236498.0488
2018-08-31,214980.4690
2018-09-30,229543.0858


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    167608.4871
 2018-02-28    119755.7206
 2018-03-31    161688.8548
 2018-04-30    164314.2814
 2018-05-31    143276.7166
 2018-06-30    168584.7850
 2018-07-31    236498.0488
 2018-08-31    214980.4690
 2018-09-30    229543.0858
 2018-10-31    167511.5855
 2018-11-30    141432.5025
 2018-12-31    136955.2799
 2019-01-31    177576.3845
 2019-02-28    154489.0498
 2019-03-31    257777.5657
 2019-04-30    203492.8705
 2019-05-31    206696.1662
 2019-06-30    196114.8983
 2019-07-31    213275.0223
 2019-08-31    306036.8783
 2019-09-30    175082.7547
 2019-10-31    179536.1484
 2019-11-30    161446.4140
 2019-12-31    189694.8902
 2020-01-31    162249.9290
 2020-02-29    147579.0425
 2020-03-31    169628.8163
 2020-04-30    153548.1563
 2020-05-31    170650.1539
 2020-06-30    168121.2907
 2020-07-31    256290.0792
 2020-08-31    237193.6724
 2020-09-30    141697.7906
 2020-10-31    120491.0838
 2020-11-30    123173.3145
 2020-12-31    142154.8782

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.984432
p-value : 0.001494
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1014.804, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1010.597, Time=0.09 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1010.069, Time=0.03 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1012.011, Time=0.07 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1013.451, Time=0.07 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1014.807, Time=0.09 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1008.810, Time=0.10 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1010.850, Time=0.04 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1012.286, Time=0.07 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1013.817, Time=0.09 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1015.466, Time=0.12 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1010.796, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1012.788, Time=0.10 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1014.231, Time=0.09 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1015.764, Time=0.12 sec
 ARIMA(3,0

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(1,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -501.398
Method:                       css-mle   S.D. of innovations          36934.692
Date:                Sat, 11 Sep 2021   AIC                           1008.797
Time:                        03:54:27   BIC                           1014.010
Sample:                             0   HQIC                          1010.708
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.751e+05   9531.661     18.366      0.000    1.56e+05    1.94e+05
ar.L1.y        0.4118      0.138      2.987      0.003       0.142       0.682
                                    Roots           

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([171265.96985239, 173496.47665985]), array([36934.69246485, 39943.69331516]), array([[ 98875.30284122, 243656.63686357],
       [ 95208.27635263, 251784.67696708]]))
